In [19]:
import tensorflow as tf
#import tensorflow_hub as hub
from google.protobuf import text_format
import numpy as np
import os
import time

gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [20]:
IMAGE_SHAPE=(224, 224)
BATCH_SIZE=32
NUM_CLASSES = 2

In [24]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255, 
    width_shift_range=.1,
    height_shift_range=.1,
    zoom_range=0.05,
    channel_shift_range=0.05,
    validation_split=0.1)
train_data = image_generator.flow_from_directory(
    "/home/gragundier/Data/ffxiv_fisher2",
    target_size=IMAGE_SHAPE,
    subset="training",
    class_mode="categorical"
    )
validation_data = image_generator.flow_from_directory(
    "/home/gragundier/Data/ffxiv_fisher2", 
    target_size=IMAGE_SHAPE,
    subset="validation",
    class_mode="categorical"
    )

Found 2469 images belonging to 2 classes.
Found 273 images belonging to 2 classes.


In [25]:
model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    # The third convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    # The final convolution (same to make compute of reshape easier)
    tf.keras.layers.Conv2D(8, (3,3), padding='same', activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    
    # Reshape to fit into LSTM/GRU
    #tf.keras.layers.Reshape((26*26,8)),
    #tf.keras.layers.LSTM(16, activation='tanh', use_bias = True, recurrent_dropout=0),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 222, 222, 128)     3584      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 111, 111, 128)     0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 111, 111, 128)     0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 109, 109, 32)      36896     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 52, 52, 16)       

In [26]:
'''
copied_metrics = [
    tfma.metrics.ExampleCount(name='example_count'),
    tfma.metrics.WeightedExampleCount(name='weighted_example_count'),
    tf.keras.metrics.SparseCategoricalCrossentropy(
        name='sparse_categorical_crossentropy'),
    tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision', top_k=1),
    tf.keras.metrics.Precision(name='precision', top_k=3),
    tf.keras.metrics.Recall(name='recall', top_k=1),
    tf.keras.metrics.Recall(name='recall', top_k=3),
    tfma.metrics.MultiClassConfusionMatrixPlot(
        name='multi_class_confusion_matrix_plot'),
]
'''
#https://www.tensorflow.org/tfx/model_analysis/metrics
#metrics_specs = tfma.metrics.specs_from_metrics(metrics)

model.compile(loss = 'categorical_crossentropy', optimizer='sgd', metrics=['accuracy', 'FalsePositives'])

In [27]:
model_name = "sequential_3"
t = time.time()
log_dir = "/home/gragundier/Data/logs/{}".format("{}-{}".format(model_name, t))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_data, epochs=500, validation_data = validation_data, callbacks=[tensorboard_callback], verbose = 1)
#history = model.fit(train_data, epochs=500, validation_data = validation_data, verbose = 1)

Epoch 1/500
78/78 [==============================] - 21s 264ms/step - loss: 0.5900 - accuracy: 0.6781 - false_positives: 374.6962 - val_loss: 0.5208 - val_accuracy: 0.6447 - val_false_positives: 97.0000
Epoch 2/500
78/78 [==============================] - 20s 251ms/step - loss: 0.3634 - accuracy: 0.8205 - false_positives: 219.7215 - val_loss: 0.7239 - val_accuracy: 0.6081 - val_false_positives: 107.0000
Epoch 3/500
78/78 [==============================] - 20s 251ms/step - loss: 0.2948 - accuracy: 0.8796 - false_positives: 154.4937 - val_loss: 0.6480 - val_accuracy: 0.6630 - val_false_positives: 92.0000
Epoch 4/500
78/78 [==============================] - 19s 249ms/step - loss: 0.2749 - accuracy: 0.8860 - false_positives: 135.0000 - val_loss: 0.7806 - val_accuracy: 0.6117 - val_false_positives: 106.0000
Epoch 5/500
78/78 [==============================] - 19s 246ms/step - loss: 0.2230 - accuracy: 0.9191 - false_positives: 102.5190 - val_loss: 0.6880 - val_accuracy: 0.6740 - val_false_po

KeyboardInterrupt: 

In [28]:
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path, save_format='tf')

export_path

INFO:tensorflow:Assets written to: /tmp/saved_models/1610732163/assets


'/tmp/saved_models/1610732163'